In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import random
import re
import os
import time
from selenium.webdriver.chrome.options import Options
import csv

### Tutorials I've used
- https://towardsdatascience.com/web-scraping-using-selenium-and-beautifulsoup-99195cd70a58
- https://www.techbeamers.com/locate-elements-selenium-python/#locate-element-by-id

## Set up selenium and chrome

In [2]:
#start selenium driver
chrome_options = Options()
#disable notifications since it blocks scrolling
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
#need headless in order to make scrolling work
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.implicitly_wait(10)

/Users/juan/Documents/manu/dev/reddit-ml/venv/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


## Scrap reddit subs
I've chosen mostly text-based subs. 
For example in science, text are more descriptive and the content is mostly a link or a video, so we save the title instead of the content based on length

In [3]:
subs = ['science','economy','Vscode','relationships','Entrepreneur','tipofmytongue','CasualConversation',]
subs = ['science','economy','Vscode']
pages = 2
random.shuffle(subs)

In [4]:
#will write posts content in a file called posts.csv
csv_file = open('posts.csv', 'w')
csv_writer = csv.writer(csv_file)
#iterate through all wanted subs
for sub in subs:
    url = "https://www.reddit.com/r/%s" % sub
    print('scrapping',url)
    #open the sub url
    driver.get(url)
    for i in range(pages):
        #scroll to bottom many times to load more posts
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #wait for the page to load
        time.sleep(3)
    #get all post links
    links = set()
    #search all links
    href_elements = driver.find_elements_by_tag_name('a')
    for href_element in href_elements:
        link =  href_element.get_attribute("href")
        #if comments is in link, means it is a post
        if 'comments' in link:
            links.add(link)
    print('links',len(links))
    #go through all links
    total = 0
    for link in links:
        print('accessing',link)
        driver.get(link)
        #get post title
        title_divs = driver.find_elements_by_tag_name('h1')
        if len(title_divs) == 0: # is'nt this a real post?
            continue
        title_div = title_divs[0]
        title = title_div.text
        #get post content
        content_divs = driver.find_elements_by_class_name('RichTextJSON-root')
        if len(content_divs) == 0: # is'nt this a real post?
            continue
        content_div = content_divs[0]
        content = content_div.text
        #what to save? title or content (we keep the lengthiest)
        #some post has a long descriptive text and no content (or video/image content)
        #so we keep title instead of post
        post_content = max([title, content], key=len)
        #if for some reason title and content are empty, skip
        if post_content.strip() == '':
            continue
        #write to csv
        post_content = post_content.replace('\n', ' ')
        csv_writer.writerow([sub,post_content])
        csv_file.flush()
        total += 1
        #I don't want to get banned
        time.sleep(5)
print('scrapped', total)


scrapping https://www.reddit.com/r/science
links 39
accessing https://www.reddit.com/r/science/comments/csij9g/nearly_200000_trans_people_have_been_exposed_to/
accessing https://www.reddit.com/r/science/comments/csj853/new_analysis_estimates_that_the_lifetime_economic/
accessing https://www.reddit.com/r/science/comments/ct2ti4/ecigarettes_change_blood_vesselsthe_study/
accessing https://www.reddit.com/r/science/comments/cscnlc/there_could_be_up_to_10_billion_warm_and_cozy/
accessing https://www.reddit.com/r/science/comments/ct07xm/new_study_shows_that_living_near_greenspace/
accessing https://www.reddit.com/r/science/comments/csigdh/scientists_detect_a_black_hole_swallowing_a/
accessing https://www.reddit.com/r/science/comments/csmvyu/a_study_published_monday_suggests_that_fluoride/
accessing https://www.reddit.com/r/science/comments/ct12a2/vaping_may_create_dangerous_toxins_that/
accessing https://www.reddit.com/r/science/comments/cswl1t/if_someone_is_trying_to_persuade_or_influence/
